<a href="https://colab.research.google.com/github/Twilight1029/Python-for-Data-Analysis/blob/main/CH_13_Introduction_to_Modeling_Libraries_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13.1 Interfacing Between pandas and Model Code

A common workflow for model development is to **use pandas for data loading and cleaning before switching over to a modeling library to build the model itself**. An important part of the model development process is called **feature engineering** in machine learning.

How to make switching between data manipulation with pandas and modeling

The point of contact between pandas and other analysis libraries is usually NumPy arrays. To turn a DataFrame into a NumPy array, use the .values property:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame({'x0' : [1, 2, 3, 4, 5],
                     'x1' : [0.01, -0.01, 0.25, -4.1, 0.],
                     'y' : [-1.5, 0., 3.6, 1.3, -2.]})

In [ ]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [ ]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [ ]:
# Use df.values to turn a df into array
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [ ]:
# convert array back to df
# pass a two-dimensional ndarray with optional column names
df2 = pd.DataFrame(data.values, columns = ['one', 'two', 'three'])

In [ ]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


The .values attribute is intended to be used when your data is **homogeneous**—for example, all numeric types. If you have heterogeneous data, the result will be an ndarray of Python objects:

In [ ]:
df3 = data.copy()

In [ ]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [ ]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [ ]:
df3.values # different type of data in df, result in object not array

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

Use **loc** indexing with values to filter or select a subset of the columns

In [ ]:
model_cols = ['x0', 'x1']

In [ ]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Categorical type and pandas.get_dummies function

In [ ]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories = ['a', 'b'])

In [ ]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


If we wanted to replace the 'category' column with dummy variables, we create dummy variables, drop the 'category' column, and then join the result:

In [ ]:
dummies = pd.get_dummies(data.category, prefix = 'category')

In [ ]:
data_with_dummies = data.drop('category', axis = 1).join(dummies)

In [ ]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# 13.2 Creating Model Descriptions with Patsy

**Patsy** is a Python library for **describing statistical models (especially linear models) with a small string-based “formula syntax,”** which is inspired by (but not exactly the same as) the formula syntax used by the R and S statistical programming languages.


Patsy is well supported for specifying linear models in statsmodels. Pasty's formulas are a special string syntax that looks like:
 $$ y \sim x_0 + x_1 $$

The syntax a + b does not mean to add a to b, but rather that these are **terms in the design matrix created for the model**. The patsy.dmatrices function takes a formula string along with a dataset (which can be a DataFrame or a dict of arrays) and produces design matrices for a linear model:

In [ ]:
data = pd.DataFrame({'x0' : [1, 2, 3, 4, 5],
                     'x1' : [0.01, -0.01, 0.25, -4.1, 0.],
                     'y' : [-1.5, 0., 3.6, 1.3, -2.]})

In [ ]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [ ]:
import patsy

In [ ]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [ ]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [ ]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These Patsy DesignMatrix instances are NumPy ndarrays with additional metadata:

In [ ]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [ ]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

**Intercept term came fro the convention for linear models like ordinary least squares (OLS) regression**. You can suppress the intercept by adding the **term + 0** to the model:

In [ ]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

The Patsy objects can be passed directly into algorithms like **numpy.linalg.lstsq, which performs an ordinary least squares regression:**

In [ ]:
# Return the least-squares solution to a linear matrix equation.
coef, resid, _, _ = np.linalg.lstsq(X, y) # what does _ means here

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [ ]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [ ]:
# a.squeeze()->remove single-dimensional entries from the shape of 'a'
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)

In [ ]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## Data Transformations in Patsy Formulas

We can mix python code into Patsy formulas, when evaluating the formula the library will try to find the functions you use in the enclosing scope

In [ ]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [ ]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Some commonly used variable transformations include **standardizing (to mean 0 and variance 1) and centering (subtracting the mean)**. Patsy has built-in functions for this purpose:

In [ ]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [ ]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

The **patsy.build_design_matrices** function can **apply transformations to new out-of-sample data using the saved information from the original in-sample dataset:**

In [ ]:
new_data = pd.DataFrame({
    'x0' : [6, 7,8, 9],
    'x1' : [3.1, -0.5, 0, 2.3],
    'y' : [1, 2, 3, 4]
})

In [ ]:
# pass the deisgn_info of X to new_X
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [ ]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Because the plus symbol (+) in the context of Patsy formulas does not mean addition, when you want to add columns from a dataset by name, you must wrap them in the special I function:

In [ ]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [ ]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Patsy has several other built-in transforms in the **patsy.builtins** module. See the online documentation for more.

## Categorical Data and Patsy

When you use non-numeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity:

In [ ]:
data = pd.DataFrame({
    'key1' : ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2' : [0, 1, 0, 1, 0, 1, 0, 0],
    'v1' : [1, 2, 3, 4, 5, 6, 7, 8],
    'v2' : [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [ ]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [ ]:
X # only 1 level in key 1, because we have intercept here

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

If you omit the intercept from the model, then columns for each category value will be included in the model design matrix:

In [ ]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [ ]:
X # we have 2 dummies now because we don't have intercept here

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Numeric columns can be interpreted as categorical with C function

In [ ]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [ ]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

When you’re using multiple categorical terms in a model, things can be more complicated, as you can **include interaction terms of the form key1:key2**, which can be used, for example, **in analysis of variance (ANOVA) models**:

In [ ]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [ ]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [ ]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [ ]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [ ]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [ ]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.3 Introduction to statsmodels

**statsmodels** is a Python library for **fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization**. 

Statsmodels contains more “classical” frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.
Some kinds of models found in statsmodels include:
* Linear models, generalized linear models, and robust linear models
* Linear mixed effects models
* Analysis of variance (ANOVA) methods
* Time series processes and state space models
* Generalized method of moments

## Estimating Linear Models

Linear models in statsmodels have two different main interfaces: **array-based and formula-based.** These are accessed through these API module imports:

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# generate a linear model from some random data
def dnorm(mean, variance, size = 1):
  if isinstance(size, int):
    size = size,
  return mean + np.sqrt(variance) * np.random.randn(*size)

In [ ]:
np.random.seed(12345)

In [ ]:
N = 100
X = np.c_[dnorm(0, 0.4, size = N),
          dnorm(0, 0.6, size = N),
          dnorm(0, 0.2, size = N)]
eps = dnorm(0, 0.1, size = N)
beta = [0.1, 0.3, 0.5]

In [ ]:
y = np.dot(X, beta) + eps

In [ ]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [ ]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

A linear model is generally fitted with an intercept term as we saw before with Patsy. The **sm.add_constant** function can add an intercept column to an existing matrix:

In [ ]:
X_model = sm.add_constant(X)

In [ ]:
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

The **sm.OLS** can fit an ordinary least squares linear regression:

In [ ]:
model = sm.OLS(y, X)

The model’s fit method returns a regression results object containing estimated model parameters and other diagnostics:

In [ ]:
results = model.fit()

In [ ]:
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [ ]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Mon, 18 Jan 2021   Prob (F-statistic):                    7.44e-12
Time:                        13:39:50   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Suppose all of the model parameters are in a DataFrame:

In [ ]:
data = pd.DataFrame(X, columns = ['col0', 'col1', 'col2'])

In [ ]:
data['y'] = y

In [ ]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [ ]:
# Create a Model from a formula and dataframe.
results = smf.ols('y ~ col0 + col1 + col2', data).fit()

In [ ]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [ ]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

Observe how statsmodels has returned results as Series with the DataFrame column names attached. We also do not need to use add_constant when using formulas and pandas objects. The sm/smf function will automatically generate intercept values.

Given new out-of-sample data, you can compute predicted values given the estimated model parameters:

In [ ]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

## Estimating Time Series Processes

Let's simulate some time series data with an autoregressive structure and noise:

In [ ]:
init_x = 4

In [ ]:
import random
values = [init_x, init_x]
N = 1000

In [ ]:
b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
  new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
  values.append(new_x)

This data has an AR(2) structure (two lags) with parameters 0.8 and –0.4. When you fit an AR model, you may not know the number of lagged terms to include, so you can fit the model with some larger number of lags:

In [ ]:
MAXLAGS = 5

In [ ]:
model = sm.tsa.AR(values)

In [ ]:
results = model.fit(MAXLAGS)

The estimated parameters in the results have the intercept first and the estimates for the first two lags next:

In [ ]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

# 13.4 Introduction to scikit-learn

scikit-learn is one of the most widely used and trusted general-purpose Python machine learning toolkits. It contains a broad selection of **standard supervised and unsupervised machine learning methods with tools for model selection and evaluation, data transformation, data loading, and model persistence. These models can be used for classification, clustering, prediction, and other common tasks.**

Use a now-classic dataset from a Kaggle competition about passenger survival rates on the Titanic, which sank in 1912. We load the test and training dataset using pandas:

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [ ]:
# look at the columns to check for missing data
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In statistics and machine learning examples like this one, a typical task is to predict whether a passenger would survive based on features in the data. A model is fitted on a training dataset and then evaluated on an out-of-sample testing dataset.

Use Age as a predictor, but we need to first deal with missing values. There are a number of ways to do missing data imputation, but I will do a simple one and use the median of the training dataset to fill the nulls in both tables:

In [ ]:
impute_value = train['Age'].median()

In [ ]:
train['Age'] = train['Age'].fillna(impute_value)

In [ ]:
test['Age'] = test['Age'].fillna(impute_value)

Now we need to specify our models. Add a column IsFemale as an encoded version of Sex column

In [ ]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [ ]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

Then we decide on some model variables and create Numpy arrays

In [ ]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [ ]:
X_train = train[predictors].values

In [ ]:
X_test = test[predictors].values

In [ ]:
y_train = train['Survived'].values

In [ ]:
X_train[:2]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.]])

In [ ]:
y_train[:2]

array([0, 1])

We use the LogisticRegression model from scikit-learn and create a model instance:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
y_predict[: 10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

Compute an accuracy per‐ centage or some other error metric:

Many models have parameters that can be tuned, and there are techniques such as cross-validation that can be used for parameter tuning to avoid overfitting to the training data. This can often yield better predictive performance or robustness on new data.

Cross-validation works by splitting the training data to simulate out-of-sample pre‐ diction. Based on a model accuracy score like mean squared error, one can perform a grid search on model parameters. Some models, like logistic regression, have estima‐ tor classes with built-in cross-validation. For example, the LogisticRegressionCV class can be used with a parameter indicating how fine-grained of a grid search to do on the model regularization parameter C:

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
model_cv = LogisticRegressionCV(10)

In [ ]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

To do cross-validation by hand, you can use the cross_val_score helper function, which handles the data splitting process. For example, to cross-validate our model with four non-overlapping splits of the training data, we can do:

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
model = LogisticRegression(C = 10)

In [ ]:
scores = cross_val_score(model, X_train, y_train, cv = 4)

In [ ]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])